In [2]:
import numpy as np
import sys
sys.path.append('/Users/krabchuk/tmp')
from ml_basics.utils.general import split

In [3]:
from sklearn import datasets
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [4]:
X, y = datasets.load_digits(return_X_y=True)
X_train, y_train, X_test, y_test = split(X, y, 0.8)

In [39]:
import numpy as np

def node_split(d, t, i, v, return_data=False):
    if isinstance(v, (int, float, np.int32, np.int64, np.float32, np.float64)):
        m = d[:,i] < v
    elif type(v) is str:
        m = d[:,i] == v
    else:
        raise TypeError(f"Unknown type {type(v)} for split value {v}")
        
    if not return_data:
        return t[m], t[~m]
    else:
        return d[m], t[m], d[~m], t[~m]
    
def calc_gini(d, t, i, v):
    left, right = node_split(d, t, i, v)
    
    gini_left = 1 - ((np.unique(left, return_counts=True)[1] / len(left)) ** 2).sum()
    gini_right = 1 - ((np.unique(right, return_counts=True)[1] / len(right)) ** 2).sum()

    
    return (gini_left * len(left) + gini_right * len(right)) / len(t)

def best_split(d, t):
    i_best = 0
    v_best = 0
    gini_best = 1
    for i in range(d.shape[1]):
        values = np.unique(d[:,i])
        for v in values:
            gini = calc_gini(d, t, i, v)
            if gini < gini_best:
                i_best = i
                v_best = v
                gini_best = gini
    return i_best, v_best

class DecisionNode:
    def __init__(self, i, v, max_depth, num_classes):
        self.i = i
        self.v = v
        self.is_leaf = False
        self.max_depth = max_depth
        self.num_classes = num_classes
            
    def build_children(self, d, t, curr_depth):
        if curr_depth == self.max_depth or len(np.unique(t)) == 1:
            self.is_leaf = True
            v, c = np.unique(t, return_counts=True)
            self.t = np.zeros(self.num_classes)
            self.t[v] = c / len(t)
            return
        
        d_l, t_l, d_r, t_r = node_split(d, t, self.i, self.v, return_data=True)
        
        self.left = DecisionNode(*best_split(d_l, t_l), self.max_depth, self.num_classes)
        self.left.build_children(d_l, t_l, curr_depth + 1)
        
        self.right = DecisionNode(*best_split(d_r, t_r), self.max_depth, self.num_classes)
        self.right.build_children(d_r, t_r, curr_depth + 1)
        
        
    def chose_child(self, d):
        if type(self.v) is str:
            return self.left if d[self.i] == self.v else self.right
        else:
            return self.left if d[self.i] < self.v else self.right
        
        
    def get_t(self, d):
        if self.is_leaf:
            return self.t
        
        child = self.chose_child(d)
        return child.get_t(d)

class DecisionTree:
    def __init__(self, depth=7, num_classes=10):
        self.depth = depth
        self.num_classes = num_classes
    
    def fit(self, data, target):
        i, v = best_split(data, target)
        self.root = DecisionNode(i, v, self.depth, self.num_classes)
        self.root.build_children(data, target, 0)
        
    def predict(self, data):
        return list(map(lambda x: self.root.get_t(x), data))

In [40]:
m = DecisionTree(5, len(np.unique(y_test)))
m.fit(X_train, y_train)
m.predict(X_test)
accuracy_score(y_test, np.argmax(m.predict(X_test), axis=1))

0.6906077348066298

In [36]:
from sklearn.tree import DecisionTreeClassifier

In [37]:
m_v = DecisionTreeClassifier()
m_v.fit(X_train, y_train)
accuracy_score(y_test, m_v.predict(X_test))

0.8619718309859155

In [25]:
x = np.zeros(10)
x[[0,2]] = 1
x

array([1., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
class GBTrees:
    def __init__(self, n_classes, depth, iters):
        self.wl = []
        self.n_classes = n_classes
        self.depth = depth
        self.iters = iters
        
    def fit(self, X, y):
        for i in range(self.iters):
            m = DecisionTree(self.depth, self.n_classes)
            m.fit(X[m], y[m])
            weak_learners.append(m)
            pred = m.predict(custom_X[m]) 
            
            
    def predict(self, X):
        pred = np.zeros(self.n_classes)
        for i in range(self.iters):
            pred += self.wl[i].predict(X)
        return np.argmax(pred)

In [47]:
1. / (1 + np.exp(-10.535552))

2.657396517418171e-05

In [ ]:
weak_learners = []
custom_X = X_train
custom_y = y_train
total_pred = None
m = np.ones(len(y_train))
for i in range(10):
    m = DecisionTree(5, len(np.unique(y_test)))
    m.fit(custom_X[m], custom_y[m])
    weak_learners.append(m)
    pred = m.predict(custom_X[m])
    total_pred += 
    
    
    
    
    print(accuracy_score(y_test, pred)